In [6]:
from utils.binaries import *

import pickle

In [17]:
import uproot

def get_start_stop_variance(path):
    path='../NightSkyBackground'
    file = uproot.open(f'{path}/{file}:AugerUpTime')
    variances = np.array(file['ADCVariance'].array(), dtype=float)[:,-3:,:]
    shutter_open_fraction = np.array(file['ShutterFraction8ADC'].array())[:,-3:]
    variances *= shutter_open_fraction[:, :, np.newaxis]

    indices = [i for i, var in enumerate(variances) if np.sum(var) != 0]
    start, stop = file['uptime_gpsStart'].array(), file['uptime_gpsStop'].array()
    return start[indices], stop[indices], variances[indices]

def get_calibration_constant(date, method='xy'):

    results = '/cr/data01/filip/xy-calibration/results/'

    # force preclean usage
    # date = '2023-07'

    # force postclean usage
    # date = '2023-11'

    if date < '2023-11':        # before mirror cleaning
        heat1_run = '6020x'
        heat2_run = '6025'
        heat3_run = '6031x'

    else:                       # after mirror cleaning
        heat1_run = '6070'
        heat2_run = '6075'
        heat3_run = '6066'

    heat1 = pd.read_csv(f'{results}/outCorr_{heat1_run}.txt', skiprows=5, index_col=0, delimiter=' ',
                        names=['pixel', 'calibConst', 'adcSum', 'stdCalib', 'calibConstCorr'])
    heat2 = pd.read_csv(f'{results}/outCorr_{heat2_run}.txt', skiprows=5, index_col=0, delimiter=' ',
                        names=['pixel', 'calibConst', 'adcSum', 'stdCalib', 'calibConstCorr'])
    heat3 = pd.read_csv(f'{results}/outCorr_{heat3_run}.txt', skiprows=5, index_col=0, delimiter=' ',
                        names=['pixel', 'calibConst', 'adcSum', 'stdCalib', 'calibConstCorr'])
    
    match method.lower():
        case 'xy':
            calibration_constants = heat1.calibConstCorr.values, heat2.calibConstCorr.values, heat3.calibConstCorr.values
        case 'cala':
            calibration_constants = heat1.stdCalib.values, heat2.stdCalib.values, heat3.stdCalib.values
        case _:
            calibration_constants = np.ones((3, 440))

    return calibration_constants

In [22]:
def do_all_things(file):

    with open('gps-second-dict_2023-10-04_2023-11-03.pickle', 'rb') as f: time = pickle.load(f)['Heat']


    path='../NightSkyBackground'
    file = uproot.open(f'{path}/{file}:AugerUpTime')

    fine_start = min([(time[f'TelID: {i}']['gps-second']).min() for i in range(1,4)])
    fine_stop = max([(time[f'TelID: {i}']['gps-second']).max() for i in range(1,4)])
    start, stop = file['uptime_gpsStart'].array(), file['uptime_gpsStop'].array()
    rough_start, rough_stop = min(start), max(stop)

    start, stop = max(fine_start, rough_start), min(fine_stop, rough_stop)

    # get bandwidth/bandwidth
    with open('bandwidth-dict_2023-10-04_2023-11-03.pickle', 'rb') as f: bandwidth = pickle.load(f)['Heat']
    with open('gain-dict_2023-10-04_2023-11-03.pickle', 'rb') as f: gain = pickle.load(f)['Heat']

    print(bandwidth)

    # get variance
    variances = np.array(file['ADCVariance'].array(), dtype=float)[:,-3:,:]
    print(start, stop)

In [23]:
do_all_things('2023-11-01-12utc.root')

{'TelID: 1': {'bandwidth': array([[5120000., 4900000., 4980000., ..., 4840000., 4940000., 4950000.],
       [5120000., 4900000., 4980000., ..., 4840000., 4940000., 4950000.],
       [5120000., 4900000., 4980000., ..., 4840000., 4940000., 4950000.],
       ...,
       [5120000., 4900000., 4980000., ..., 4840000., 4940000., 4950000.],
       [5120000., 4900000., 4980000., ..., 4840000., 4940000., 4950000.],
       [5120000., 4900000., 4980000., ..., 4840000., 4940000., 4950000.]])}, 'TelID: 2': {'bandwidth': array([[4950000., 4950000., 4890000., ..., 4970000., 4760000., 4830000.],
       [4950000., 4950000., 4890000., ..., 4970000., 4760000., 4830000.],
       [4950000., 4950000., 4890000., ..., 4970000., 4760000., 4830000.],
       ...,
       [4950000., 4950000., 4890000., ..., 4970000., 4760000., 4830000.],
       [4950000., 4950000., 4890000., ..., 4970000., 4760000., 4830000.],
       [4950000., 4950000., 4890000., ..., 4970000., 4760000., 4830000.]])}, 'TelID: 3': {'bandwidth': arr

In [ ]:
from datetime import datetime
from utils.Auger.FD import PixelPlot
from matplotlib.gridspec import GridSpec

files = [#'2022-10-01-12utc.root',
        #  '2023-03-01-12utc.root',
        #  '2023-04-01-12utc.root',
        #  '2023-05-01-12utc.root',
        #  '2023-06-01-12utc.root',
        #  '2023-07-01-12utc.root', 
        #  '2023-08-01-12utc.root', 
        #  '2023-09-01-12utc.root', 
        #  '2023-10-01-12utc.root', 
         '2023-11-01-12utc.root', 
         '2023-12-01-12utc.root',
        #  '2024-01-01-12utc.root'
         ]

fig1, laxes = plt.subplots(len(files), 1, sharey=True)
fig1.text(.02, 0.5, 'Variance / $\mathrm{ADC}^2$', rotation=90,
         fontdict={'ha' : 'center', 'va' : 'center', 'fontsize' : 18})
plt.subplots_adjust(hspace=0.6)

fig2 = plt.figure()
gs = GridSpec(
        3,
        len(files) + 1,
        figure=fig2,
        width_ratios=[1 for _ in files] + [0.1],
        height_ratios = [1/3, 1/3, 1/3],
    )
gs.update(left=0.05, right=0.95, wspace=0.05, hspace=0.02)

fig2.text(.02, 0.76, 'Heat #1', rotation=90,
          fontdict={'ha' : 'center', 'va' : 'center', 'fontsize' : 9})
fig2.text(.02, 0.5, 'Heat #2', rotation=90,
          fontdict={'ha' : 'center', 'va' : 'center', 'fontsize' : 9})
fig2.text(.02, 0.24, 'Heat #3', rotation=90,
          fontdict={'ha' : 'center', 'va' : 'center', 'fontsize' : 9})

method = 'xy'
if method == '_':
    vmin, vmax = 0, 50
    name = 'raw_variance'
elif method == 'cala':
    vmin, vmax = 30, 140
    name = 'CalA_normalized'
elif method == 'xy':
    vmin, vmax = 30, 140
    name = 'XY_normalized'

means_before, means_after = [], []

for col, (file, lax) in enumerate(zip(files, laxes)):

    flux = do_all_things(file)

    start, stop, var = get_start_stop_variance(file)

    skip = 35 if file == '2022-10-01-12utc.root' else 0
    timestamps = [datetime.fromtimestamp(0.5*(e+b)) for e, b in zip(start, stop)]
    lax.set_title(file[:7], pad=1, fontsize=6)
    lax.set_xticklabels([])

    lax.scatter(timestamps[skip:], np.mean(var[skip:,0,:], axis=1), label='Heat #1', s=0.1)
    lax.scatter(timestamps[skip:], np.mean(var[skip:,1,:], axis=1), label='Heat #2', s=0.1, zorder=10)
    lax.scatter(timestamps[skip:], np.mean(var[skip:,2,:], axis=1), label='Heat #3', s=0.1)
    lax.legend(ncols=3, fontsize=5)

    ax1, ax2, ax3 = [fig2.add_subplot(gs[i,col]) for i in range(3)]
    means = [np.median(var[:, i,:], axis=0) for i in range(3)]

    # normalize with calibration constants
    calibration_constants = get_calibration_constant(file[:7], method)
    means = np.array(means) * calibration_constants # * K_V?

    PixelPlot(means[0], ax=ax1, lw=0.3, vmin=vmin, vmax=vmax)
    PixelPlot(means[1], ax=ax2, lw=0.3, vmin=vmin, vmax=vmax)
    PixelPlot(means[2], ax=ax3, lw=0.3, vmin=vmin, vmax=vmax)
    ax1.set_title(file[:7], pad=4, fontsize=9)

    if file[:7] < '2023-11':
        if file[:7] in ['2023-09', '2023-07']:                      # skip heat 2 due to UpTime being down
            heat1, heat2, heat3 = np.mean(means, axis=1)
            heat2 = np.nan
            means_before.append([heat1, heat2, heat3])        
        else:
            means_before.append(np.mean(means, axis=1))
    else:
        means_after.append(np.mean(means, axis=1))

    ax1.text(0.09, 0.93, fr'$\langle\mathrm{{\sigma^2_\mathrm{{FADC}}}}\rangle = {np.mean(means[0]):.2f}$', c='white', fontdict={'fontsize' : 5, 'ha' : 'left', 'va' : 'top', 'weight' : 'bold'}, transform=ax1.transAxes)
    ax2.text(0.09, 0.93, fr'$\langle\mathrm{{\sigma^2_\mathrm{{FADC}}}}\rangle = {np.mean(means[1]):.2f}$', c='white', fontdict={'fontsize' : 5, 'ha' : 'left', 'va' : 'top', 'weight' : 'bold'}, transform=ax2.transAxes)
    ax3.text(0.09, 0.93, fr'$\langle\mathrm{{\sigma^2_\mathrm{{FADC}}}}\rangle = {np.mean(means[2]):.2f}$', c='white', fontdict={'fontsize' : 5, 'ha' : 'left', 'va' : 'top', 'weight' : 'bold'}, transform=ax3.transAxes)

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
norm = Normalize(vmin, vmax)

fig2.colorbar(ScalarMappable(norm, 'viridis'), cax=fig2.add_subplot(gs[:, len(files)]),
              label=r'$\langle\mathrm{\sigma^2_\mathrm{{FADC}}}\rangle$ / $\mathrm{ADC}^2$')